# Fetal Health Classification
### Assess the well being of a fetus undergoing a CTG exam

## Task Details
### Create a model to classify the outcome of Cardiotocogram (CTG) exam (which represents the well being of the fetus).

## Notes
#### Note that this is a multiclass problem that can also be treated as regression (since the labels are progressive).
#### Also note that there is considerable class imbalance and therefore accuracy is not recommended as primary metric and stratification is recommended when splitting the data.

## Evaluation
### With a testing set of size of 30% of all available data, calculate some of the recommended metrics bellow:

   1. Area under the ROC Curve
   2. F1 Score
   3. Area under the Precision-Recall Curve

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from warnings import filterwarnings
filterwarnings(action='ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/fetal-health-classification/fetal_health.csv")
data.shape

In [ ]:
data.head()
print(data.describe())
print(data.info())

In [ ]:
# Understand the correlation between differnt variables in the dataset
corr = data.corr()
plt.figure(figsize=(20, 12))
sns.heatmap(corr, cmap="coolwarm", annot=True)

In [ ]:
data['fetal_health'] = data['fetal_health'].astype('int')
data['fetal_health'].value_counts()

In [ ]:
''' Creating Bar plot for our target variable '''
plt.figure(figsize=(15,5))
data.fetal_health.value_counts().plot(kind='bar')
plt.xlabel('Health Status')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
''' Visualising through Pie chart as well '''
data.fetal_health.value_counts().plot(kind='pie')

### Since there is large imbalance of classes in our data set, we can perform over sampling or down sampling.

In [ ]:
""" Before Over Sampling """
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
print(X.shape,y.shape)
print(Counter(y))

In [ ]:
''' After Over Sampling '''
# !pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
print(X.shape,y.shape)
print(Counter(y))

### We can perform Feature scaling on our predictor variabes since they all are of numerical data type.

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve,roc_auc_score,f1_score,precision_recall_curve,confusion_matrix,classification_report

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.20,shuffle=True,stratify=y,random_state = 42)

In [ ]:
lr_model = LogisticRegression(penalty='l2',dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=42,
    solver='lbfgs',
    max_iter=100,
    multi_class='ovr',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None,)

In [ ]:
y_pred_train = lr_model.fit(x_train,y_train)

y_pred_test = lr_model.predict(x_test)

In [ ]:
cm = confusion_matrix(y_test,y_pred_test)
cm

In [ ]:
report = classification_report(y_test,y_pred_test)
print(report)

## Thus, with the help of over sampling technique,  we have successfully improved our Precision and Recall Scores for our different classes in target variable.

### We can also use other enhanced Classifiers as well for improving our scores.